In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
path='D:/Python/ML Starter Program'

In [3]:
train=pd.read_csv(path+'/'+'train_HK6lq50.csv')
test=pd.read_csv(path+'/'+'test_wF0Ps6O.csv')
sample_submission=pd.read_csv(path+'/'+'sample_submission_vaSxamm.csv')

In [4]:
train.columns

Index(['id', 'program_id', 'program_type', 'program_duration', 'test_id',
       'test_type', 'difficulty_level', 'trainee_id', 'gender', 'education',
       'city_tier', 'age', 'total_programs_enrolled', 'is_handicapped',
       'trainee_engagement_rating', 'is_pass'],
      dtype='object')

In [5]:
test["is_pass"]=0
train["Temp"]="train"
test["Temp"]="test"
combi=pd.concat([train,test],axis=0,ignore_index=True)

In [6]:
print(combi.shape)
print(combi.dtypes)
combi.isnull().sum(axis=0)

(104496, 17)
id                            object
program_id                    object
program_type                  object
program_duration               int64
test_id                        int64
test_type                     object
difficulty_level              object
trainee_id                     int64
gender                        object
education                     object
city_tier                      int64
age                          float64
total_programs_enrolled        int64
is_handicapped                object
trainee_engagement_rating    float64
is_pass                        int64
Temp                          object
dtype: object


id                               0
program_id                       0
program_type                     0
program_duration                 0
test_id                          0
test_type                        0
difficulty_level                 0
trainee_id                       0
gender                           0
education                        0
city_tier                        0
age                          39520
total_programs_enrolled          0
is_handicapped                   0
trainee_engagement_rating      108
is_pass                          0
Temp                             0
dtype: int64

In [7]:
categorical_col=[x for x in combi.dtypes.index if combi.dtypes[x]=="object"]
categorical_2=[x for x in categorical_col if x not in ["id","program_id","Temp"]]
for i in categorical_2:
    print("Frequency of Category %s"%i)
    print(combi[i].value_counts(dropna=False))

Frequency of Category program_type
Y    32304
T    26361
V    18594
U    11213
Z     9337
X     4728
S     1959
Name: program_type, dtype: int64
Frequency of Category test_type
offline    62256
online     42240
Name: test_type, dtype: int64
Frequency of Category difficulty_level
easy            57211
intermediate    29537
hard            14777
vary hard        2971
Name: difficulty_level, dtype: int64
Frequency of Category gender
M    55873
F    48623
Name: gender, dtype: int64
Frequency of Category education
High School Diploma    48983
Matriculation          37432
Bachelors              16590
Masters                  754
No Qualification         737
Name: education, dtype: int64
Frequency of Category is_handicapped
N    95247
Y     9249
Name: is_handicapped, dtype: int64


In [8]:
combi["trainee_engagement_rating"] = combi.groupby(["program_id","difficulty_level"])["trainee_engagement_rating"].transform(lambda x: x.fillna(x.median()))

In [9]:
nunique_program_id= pd.pivot_table(combi,values=['program_id'],index='trainee_id',aggfunc=pd.Series.nunique)

In [10]:
combi=pd.merge(combi,nunique_program_id,on=['trainee_id'],how='left')

In [11]:
num_of_test=pd.pivot_table(combi,values=['test_id'],index='trainee_id',aggfunc=pd.Series.nunique)

In [12]:
combi=pd.merge(combi,num_of_test,on=['trainee_id'],how='left')

In [13]:
combi.rename(columns={"program_id_x":"program_id","test_id_x":"test_id","program_id_y": "nunique_program_id","test_id_y":"num_of_test"},inplace=True)

In [14]:
combi.describe(percentiles=[0.01,0.05,0.95,0.99]).T

,count,mean,std,min,1%,5%,50%,95%,99%,max
program_duration,104496.0,128.209386,6.892792,117.0,117.00,117.00,131.0,136.0,136.0,136.0
test_id,104496.0,90.937672,51.219793,0.0,5.00,14.00,90.0,171.0,184.0,187.0
trainee_id,104496.0,9846.168045,5706.697072,1.0,216.95,1036.75,9648.5,19059.0,19891.0,20098.0
city_tier,104496.0,2.248478,1.011789,1.0,1.00,1.00,2.0,4.0,4.0,4.0
age,64976.0,36.478038,9.045325,17.0,21.00,24.00,39.0,48.0,56.0,63.0
total_programs_enrolled,104496.0,2.586223,1.238415,1.0,1.00,1.00,2.0,5.0,7.0,14.0
trainee_engagement_rating,104496.0,2.396972,1.326923,1.0,1.00,1.00,2.0,5.0,5.0,5.0
is_pass,104496.0,0.486784,0.499828,0.0,0.00,0.00,0.0,1.0,1.0,1.0
nunique_program_id,104496.0,1.158035,0.386160,1.0,1.00,1.00,1.0,2.0,2.0,4.0
num_of_test,104496.0,7.180179,3.113487,1.0,1.00,2.00,7.0,12.0,16.0,23.0


In [15]:
combi.loc[combi["age"]>combi["age"].quantile(0.99),"age"]=combi["age"].quantile(0.99)
combi.loc[combi["age"]<combi["age"].quantile(0.01),"age"]=combi["age"].quantile(0.01)
combi.loc[combi["total_programs_enrolled"]>combi["total_programs_enrolled"].quantile(0.99),"total_programs_enrolled"]=combi["total_programs_enrolled"].quantile(0.99)
combi.loc[combi["nunique_program_id"]>combi["nunique_program_id"].quantile(0.99),"nunique_program_id"]=combi["nunique_program_id"].quantile(0.99)
combi.loc[combi["num_of_test"]>combi["num_of_test"].quantile(0.99),"num_of_test"]=combi["num_of_test"].quantile(0.99)

In [16]:
dummy=pd.get_dummies(combi[["program_type","difficulty_level","gender","education","city_tier","is_handicapped","test_type"]],drop_first=False)

In [17]:
mydata=pd.concat([combi,dummy],axis=1)

In [18]:
mydata.drop(["program_type","difficulty_level","gender","education","city_tier","is_handicapped","test_type","test_id","id","trainee_id","program_id"],inplace=True,axis=1)

In [19]:
### Impute Missing values in age by using other variables, will use KNN Regressor for that, creating training and testing data ##
subset1 = mydata.loc[combi["age"].notnull()]
subset2=mydata.loc[combi["age"].isnull()]

In [20]:
subset2.drop(["age","Temp"],inplace=True,axis=1)

In [21]:
train=subset1.drop(["age","Temp"],axis=1)
test=subset1["age"]

In [22]:
from sklearn.model_selection import train_test_split

In [23]:
np.random.seed(324)

In [24]:
x_train,x_test,y_train,y_test=train_test_split(train,test, test_size=0.25,random_state=43)

In [5]:
#!jt -t grade3

In [25]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt
from random import randint
from sklearn.model_selection import RandomizedSearchCV

In [26]:
# param_dist_rf = {"n_estimators":[10,100,500,700,1000],
#               "max_depth": [6,8,10,12, None],
#               "max_features": [i/10.0 for i in range(6,10)],
#               "bootstrap": [True, False]}

In [27]:
# model=RandomForestRegressor()

In [28]:
# random_search_rf = RandomizedSearchCV(model, param_distributions=param_dist_rf,
#                               n_iter=10)

In [29]:
# random_search_rf.fit(x_train,y_train)

In [30]:
# def report(results, n_top=3):
#     for i in range(1, n_top + 1):
#         candidates = np.flatnonzero(results['rank_test_score'] == i)
#         for candidate in candidates:
#             print("Model with rank: {0}".format(i))
#             print("Mean validation score: {0:.3f} (std: {1:.5f})".format(
#                   results['mean_test_score'][candidate],
#                   results['std_test_score'][candidate]))
#             print("Parameters: {0}".format(results['params'][candidate]))
#             print("")

In [31]:
# report(random_search_rf.cv_results_,5)

In [32]:
model=RandomForestRegressor(n_estimators=500, max_features=0.6, max_depth=None, bootstrap=True)

In [33]:
model.fit(x_train,y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=0.6, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [34]:
## RF RMSE ###
sqrt(mean_squared_error(y_test,np.round(model.predict(x_test))))

6.9765675270503

In [35]:
subset2["age"]=np.round(model.predict(subset2))

In [36]:
mydata_1=pd.concat([subset1,subset2],axis=0, ignore_index=False)

In [37]:
mydata_2=mydata_1.sort_index()

In [38]:
mydata["age"]=mydata_2["age"]

In [39]:
mydata.to_csv("mydata.csv",index=False)

In [40]:
training=mydata.loc[mydata["Temp"]=="train"]
testing=mydata.loc[mydata["Temp"]=="test"]

In [41]:
x_train=training.drop(["Temp","is_pass"],axis=1)
y_train=training["is_pass"]
testing.drop(["Temp","is_pass"],axis=1,inplace=True)

In [42]:
np.random.seed(324)

In [43]:
from sklearn.model_selection import train_test_split

In [44]:
X_train,X_test,Y_train,Y_test=train_test_split(x_train,y_train,test_size=0.25,stratify=y_train)

In [45]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.5f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [83]:
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV

In [89]:
# param_dist = {
#               "max_depth": [6,8,10,12],
#               "learning_rate":[0.1],
#               "subsample":[i/10.0 for i in range(6,10)],
#  "colsample_bytree":[i/10.0 for i in range(6,10)],
#                "reg_alpha":[1e-5, 1e-2, 0.1, 1, 100],
#               "gamma":[i/10.0 for i in range(0,5)],
#     "n_estimators":[100,500,700,1000],
#     'scale_pos_weight':[2,3,4,5,6,8,9]
    
# #               }

In [90]:
# clf=XGBClassifier(objective='binary:logistic')

In [91]:
# n_iter=10

# random_search=RandomizedSearchCV(clf,n_jobs=-1,verbose=2,cv=5,n_iter=n_iter,scoring='roc_auc',
#                                  param_distributions=param_dist)

In [92]:
# random_search.fit(X_train,Y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed: 10.2min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed: 19.4min finished


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
          estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1),
          fit_params=None, iid='warn', n_iter=10, n_jobs=-1,
          param_distributions={'max_depth': [6, 8, 10, 12], 'learning_rate': [0.1], 'subsample': [0.6, 0.7, 0.8, 0.9], 'colsample_bytree': [0.6, 0.7, 0.8, 0.9], 'reg_alpha': [1e-05, 0.01, 0.1, 1, 100], 'gamma': [0.0, 0.1, 0.2, 0.3, 0.4], 'n_estimators': [100, 500, 700, 1000], 'scale_pos_weight': [2, 3, 4, 5, 6, 8, 9]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=

In [93]:
# report(random_search.cv_results_,5)

Model with rank: 1
Mean validation score: 0.785 (std: 0.00349)
Parameters: {'subsample': 0.7, 'scale_pos_weight': 6, 'reg_alpha': 1e-05, 'n_estimators': 700, 'max_depth': 12, 'learning_rate': 0.1, 'gamma': 0.3, 'colsample_bytree': 0.8}

Model with rank: 2
Mean validation score: 0.783 (std: 0.00326)
Parameters: {'subsample': 0.6, 'scale_pos_weight': 4, 'reg_alpha': 0.01, 'n_estimators': 700, 'max_depth': 12, 'learning_rate': 0.1, 'gamma': 0.0, 'colsample_bytree': 0.8}

Model with rank: 3
Mean validation score: 0.781 (std: 0.00374)
Parameters: {'subsample': 0.7, 'scale_pos_weight': 4, 'reg_alpha': 0.01, 'n_estimators': 500, 'max_depth': 12, 'learning_rate': 0.1, 'gamma': 0.2, 'colsample_bytree': 0.6}

Model with rank: 4
Mean validation score: 0.780 (std: 0.00411)
Parameters: {'subsample': 0.6, 'scale_pos_weight': 2, 'reg_alpha': 1e-05, 'n_estimators': 700, 'max_depth': 12, 'learning_rate': 0.1, 'gamma': 0.0, 'colsample_bytree': 0.6}

Model with rank: 5
Mean validation score: 0.779 (std: 

In [94]:
xgb_best=XGBClassifier(subsample=0.7,scale_pos_weight=6,reg_alpha=1e-05,n_estimators=700,
                       max_depth=12,learning_rate=0.1,gamma=0.3,colsample_bytree=0.8)

In [95]:
xgb_best.fit(X_train,Y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=0.8, gamma=0.3,
       learning_rate=0.1, max_delta_step=0, max_depth=12,
       min_child_weight=1, missing=None, n_estimators=700, n_jobs=1,
       nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=1e-05, reg_lambda=1, scale_pos_weight=6, seed=None,
       silent=None, subsample=0.7, verbosity=1)

In [96]:
p=xgb_best.predict_proba(X_test)[:,1]

In [97]:
from sklearn.metrics import roc_auc_score

In [98]:
roc_auc_score(Y_test,p)

0.7998048315709658

In [99]:
sample_submission["is_pass"]=xgb_best.predict_proba(testing)[:,1]

In [100]:
sample_submission.to_csv("D:/Python/ML Starter Program/sample_submission_xgb.csv",header =True, index=False)

In [50]:
from catboost import CatBoostClassifier

In [51]:
# model = CatBoostClassifier(class_weights=[1,2],eval_metric='AUC',loss_function="Logloss",random_seed = 23, learning_rate=0.1)
# parameters = {'depth'         : [6,8,10,12],
#                   'iterations'    : [1000,1500,2000,2500,3000,4000,5000]
#                  }

In [52]:
# random_search_2= RandomizedSearchCV(estimator=model, param_distributions = parameters, 
#                                cv = 5, n_jobs=-1)

In [53]:
# random_search_2.fit(X_train,Y_train)

0:	total: 417ms	remaining: 20m 50s
1:	total: 649ms	remaining: 16m 12s
2:	total: 887ms	remaining: 14m 46s
3:	total: 1.09s	remaining: 13m 36s
4:	total: 1.28s	remaining: 12m 46s
5:	total: 1.31s	remaining: 10m 55s
6:	total: 1.47s	remaining: 10m 29s
7:	total: 1.61s	remaining: 10m 3s
8:	total: 1.75s	remaining: 9m 42s
9:	total: 1.92s	remaining: 9m 33s
10:	total: 2.07s	remaining: 9m 21s
11:	total: 2.08s	remaining: 8m 37s
12:	total: 2.23s	remaining: 8m 33s
13:	total: 2.46s	remaining: 8m 45s
14:	total: 2.62s	remaining: 8m 41s
15:	total: 2.78s	remaining: 8m 38s
16:	total: 2.8s	remaining: 8m 11s
17:	total: 2.96s	remaining: 8m 10s
18:	total: 3.11s	remaining: 8m 8s
19:	total: 3.12s	remaining: 7m 45s
20:	total: 3.27s	remaining: 7m 43s
21:	total: 3.27s	remaining: 7m 23s
22:	total: 3.43s	remaining: 7m 24s
23:	total: 3.59s	remaining: 7m 25s
24:	total: 3.76s	remaining: 7m 27s
25:	total: 3.91s	remaining: 7m 26s
26:	total: 3.92s	remaining: 7m 12s
27:	total: 4.07s	remaining: 7m 12s
28:	total: 4.24s	remainin

233:	total: 31.5s	remaining: 6m 12s
234:	total: 31.6s	remaining: 6m 12s
235:	total: 31.8s	remaining: 6m 12s
236:	total: 31.9s	remaining: 6m 12s
237:	total: 32.1s	remaining: 6m 12s
238:	total: 32.2s	remaining: 6m 12s
239:	total: 32.4s	remaining: 6m 12s
240:	total: 32.5s	remaining: 6m 12s
241:	total: 32.7s	remaining: 6m 12s
242:	total: 32.8s	remaining: 6m 11s
243:	total: 32.9s	remaining: 6m 11s
244:	total: 33s	remaining: 6m 11s
245:	total: 33.2s	remaining: 6m 11s
246:	total: 33.3s	remaining: 6m 11s
247:	total: 33.5s	remaining: 6m 11s
248:	total: 33.6s	remaining: 6m 11s
249:	total: 33.8s	remaining: 6m 11s
250:	total: 33.9s	remaining: 6m 11s
251:	total: 34.1s	remaining: 6m 11s
252:	total: 34.2s	remaining: 6m 11s
253:	total: 34.4s	remaining: 6m 11s
254:	total: 34.5s	remaining: 6m 11s
255:	total: 34.6s	remaining: 6m 11s
256:	total: 34.8s	remaining: 6m 11s
257:	total: 34.9s	remaining: 6m 11s
258:	total: 35.1s	remaining: 6m 11s
259:	total: 35.2s	remaining: 6m 11s
260:	total: 35.4s	remaining: 6

467:	total: 1m 7s	remaining: 6m 3s
468:	total: 1m 7s	remaining: 6m 3s
469:	total: 1m 7s	remaining: 6m 2s
470:	total: 1m 7s	remaining: 6m 2s
471:	total: 1m 7s	remaining: 6m 2s
472:	total: 1m 7s	remaining: 6m 2s
473:	total: 1m 7s	remaining: 6m 2s
474:	total: 1m 8s	remaining: 6m 1s
475:	total: 1m 8s	remaining: 6m 1s
476:	total: 1m 8s	remaining: 6m 1s
477:	total: 1m 8s	remaining: 6m 1s
478:	total: 1m 8s	remaining: 6m 1s
479:	total: 1m 8s	remaining: 6m 1s
480:	total: 1m 8s	remaining: 6m 1s
481:	total: 1m 9s	remaining: 6m
482:	total: 1m 9s	remaining: 6m
483:	total: 1m 9s	remaining: 6m
484:	total: 1m 9s	remaining: 6m
485:	total: 1m 9s	remaining: 5m 59s
486:	total: 1m 9s	remaining: 5m 59s
487:	total: 1m 9s	remaining: 5m 59s
488:	total: 1m 10s	remaining: 5m 59s
489:	total: 1m 10s	remaining: 5m 59s
490:	total: 1m 10s	remaining: 5m 59s
491:	total: 1m 10s	remaining: 5m 59s
492:	total: 1m 10s	remaining: 5m 59s
493:	total: 1m 10s	remaining: 5m 58s
494:	total: 1m 10s	remaining: 5m 58s
495:	total: 1m 

691:	total: 1m 37s	remaining: 5m 26s
692:	total: 1m 37s	remaining: 5m 26s
693:	total: 1m 38s	remaining: 5m 25s
694:	total: 1m 38s	remaining: 5m 25s
695:	total: 1m 38s	remaining: 5m 25s
696:	total: 1m 38s	remaining: 5m 25s
697:	total: 1m 38s	remaining: 5m 25s
698:	total: 1m 38s	remaining: 5m 24s
699:	total: 1m 38s	remaining: 5m 24s
700:	total: 1m 38s	remaining: 5m 24s
701:	total: 1m 39s	remaining: 5m 24s
702:	total: 1m 39s	remaining: 5m 24s
703:	total: 1m 39s	remaining: 5m 24s
704:	total: 1m 39s	remaining: 5m 24s
705:	total: 1m 39s	remaining: 5m 23s
706:	total: 1m 39s	remaining: 5m 23s
707:	total: 1m 40s	remaining: 5m 23s
708:	total: 1m 40s	remaining: 5m 23s
709:	total: 1m 40s	remaining: 5m 23s
710:	total: 1m 40s	remaining: 5m 23s
711:	total: 1m 40s	remaining: 5m 22s
712:	total: 1m 40s	remaining: 5m 22s
713:	total: 1m 40s	remaining: 5m 22s
714:	total: 1m 40s	remaining: 5m 22s
715:	total: 1m 41s	remaining: 5m 22s
716:	total: 1m 41s	remaining: 5m 22s
717:	total: 1m 41s	remaining: 5m 22s
7

917:	total: 2m 9s	remaining: 4m 52s
918:	total: 2m 9s	remaining: 4m 52s
919:	total: 2m 9s	remaining: 4m 52s
920:	total: 2m 9s	remaining: 4m 52s
921:	total: 2m 9s	remaining: 4m 52s
922:	total: 2m 9s	remaining: 4m 52s
923:	total: 2m 10s	remaining: 4m 52s
924:	total: 2m 10s	remaining: 4m 52s
925:	total: 2m 10s	remaining: 4m 52s
926:	total: 2m 10s	remaining: 4m 52s
927:	total: 2m 10s	remaining: 4m 52s
928:	total: 2m 10s	remaining: 4m 51s
929:	total: 2m 11s	remaining: 4m 51s
930:	total: 2m 11s	remaining: 4m 51s
931:	total: 2m 11s	remaining: 4m 51s
932:	total: 2m 11s	remaining: 4m 51s
933:	total: 2m 11s	remaining: 4m 51s
934:	total: 2m 11s	remaining: 4m 51s
935:	total: 2m 12s	remaining: 4m 51s
936:	total: 2m 12s	remaining: 4m 51s
937:	total: 2m 12s	remaining: 4m 51s
938:	total: 2m 12s	remaining: 4m 51s
939:	total: 2m 12s	remaining: 4m 50s
940:	total: 2m 12s	remaining: 4m 50s
941:	total: 2m 13s	remaining: 4m 50s
942:	total: 2m 13s	remaining: 4m 50s
943:	total: 2m 13s	remaining: 4m 50s
944:	to

...

2232:	total: 5m 30s	remaining: 1m 53s
2233:	total: 5m 31s	remaining: 1m 53s
2234:	total: 5m 31s	remaining: 1m 53s
2235:	total: 5m 31s	remaining: 1m 53s
2236:	total: 5m 31s	remaining: 1m 53s
2237:	total: 5m 31s	remaining: 1m 52s
2238:	total: 5m 31s	remaining: 1m 52s
2239:	total: 5m 32s	remaining: 1m 52s
2240:	total: 5m 32s	remaining: 1m 52s
2241:	total: 5m 32s	remaining: 1m 52s
2242:	total: 5m 32s	remaining: 1m 52s
2243:	total: 5m 32s	remaining: 1m 52s
2244:	total: 5m 33s	remaining: 1m 52s
2245:	total: 5m 33s	remaining: 1m 51s
2246:	total: 5m 33s	remaining: 1m 51s
2247:	total: 5m 33s	remaining: 1m 51s
2248:	total: 5m 33s	remaining: 1m 51s
2249:	total: 5m 33s	remaining: 1m 51s
2250:	total: 5m 34s	remaining: 1m 51s
2251:	total: 5m 34s	remaining: 1m 51s
2252:	total: 5m 34s	remaining: 1m 50s
2253:	total: 5m 34s	remaining: 1m 50s
2254:	total: 5m 34s	remaining: 1m 50s
2255:	total: 5m 34s	remaining: 1m 50s
2256:	total: 5m 35s	remaining: 1m 50s
2257:	total: 5m 35s	remaining: 1m 50s
2258:	total:

2450:	total: 6m 11s	remaining: 1m 23s
2451:	total: 6m 11s	remaining: 1m 22s
2452:	total: 6m 11s	remaining: 1m 22s
2453:	total: 6m 11s	remaining: 1m 22s
2454:	total: 6m 11s	remaining: 1m 22s
2455:	total: 6m 12s	remaining: 1m 22s
2456:	total: 6m 12s	remaining: 1m 22s
2457:	total: 6m 12s	remaining: 1m 22s
2458:	total: 6m 12s	remaining: 1m 22s
2459:	total: 6m 13s	remaining: 1m 21s
2460:	total: 6m 13s	remaining: 1m 21s
2461:	total: 6m 13s	remaining: 1m 21s
2462:	total: 6m 13s	remaining: 1m 21s
2463:	total: 6m 13s	remaining: 1m 21s
2464:	total: 6m 14s	remaining: 1m 21s
2465:	total: 6m 14s	remaining: 1m 21s
2466:	total: 6m 14s	remaining: 1m 20s
2467:	total: 6m 14s	remaining: 1m 20s
2468:	total: 6m 15s	remaining: 1m 20s
2469:	total: 6m 15s	remaining: 1m 20s
2470:	total: 6m 15s	remaining: 1m 20s
2471:	total: 6m 15s	remaining: 1m 20s
2472:	total: 6m 15s	remaining: 1m 20s
2473:	total: 6m 16s	remaining: 1m 19s
2474:	total: 6m 16s	remaining: 1m 19s
2475:	total: 6m 16s	remaining: 1m 19s
2476:	total:

2670:	total: 6m 52s	remaining: 50.8s
2671:	total: 6m 52s	remaining: 50.7s
2672:	total: 6m 53s	remaining: 50.5s
2673:	total: 6m 53s	remaining: 50.4s
2674:	total: 6m 53s	remaining: 50.2s
2675:	total: 6m 53s	remaining: 50.1s
2676:	total: 6m 53s	remaining: 49.9s
2677:	total: 6m 54s	remaining: 49.8s
2678:	total: 6m 54s	remaining: 49.6s
2679:	total: 6m 54s	remaining: 49.5s
2680:	total: 6m 54s	remaining: 49.3s
2681:	total: 6m 54s	remaining: 49.2s
2682:	total: 6m 54s	remaining: 49s
2683:	total: 6m 55s	remaining: 48.9s
2684:	total: 6m 55s	remaining: 48.7s
2685:	total: 6m 55s	remaining: 48.6s
2686:	total: 6m 55s	remaining: 48.4s
2687:	total: 6m 55s	remaining: 48.3s
2688:	total: 6m 56s	remaining: 48.1s
2689:	total: 6m 56s	remaining: 48s
2690:	total: 6m 56s	remaining: 47.8s
2691:	total: 6m 56s	remaining: 47.7s
2692:	total: 6m 56s	remaining: 47.5s
2693:	total: 6m 57s	remaining: 47.4s
2694:	total: 6m 57s	remaining: 47.2s
2695:	total: 6m 57s	remaining: 47.1s
2696:	total: 6m 57s	remaining: 46.9s
2697:

2895:	total: 7m 38s	remaining: 16.5s
2896:	total: 7m 38s	remaining: 16.3s
2897:	total: 7m 38s	remaining: 16.1s
2898:	total: 7m 38s	remaining: 16s
2899:	total: 7m 39s	remaining: 15.8s
2900:	total: 7m 39s	remaining: 15.7s
2901:	total: 7m 39s	remaining: 15.5s
2902:	total: 7m 39s	remaining: 15.4s
2903:	total: 7m 39s	remaining: 15.2s
2904:	total: 7m 40s	remaining: 15s
2905:	total: 7m 40s	remaining: 14.9s
2906:	total: 7m 40s	remaining: 14.7s
2907:	total: 7m 40s	remaining: 14.6s
2908:	total: 7m 40s	remaining: 14.4s
2909:	total: 7m 41s	remaining: 14.3s
2910:	total: 7m 41s	remaining: 14.1s
2911:	total: 7m 41s	remaining: 13.9s
2912:	total: 7m 41s	remaining: 13.8s
2913:	total: 7m 41s	remaining: 13.6s
2914:	total: 7m 42s	remaining: 13.5s
2915:	total: 7m 42s	remaining: 13.3s
2916:	total: 7m 42s	remaining: 13.2s
2917:	total: 7m 42s	remaining: 13s
2918:	total: 7m 42s	remaining: 12.8s
2919:	total: 7m 43s	remaining: 12.7s
2920:	total: 7m 43s	remaining: 12.5s
2921:	total: 7m 43s	remaining: 12.4s
2922:	t

RandomizedSearchCV(cv=5, error_score='raise-deprecating',
          estimator=<catboost.core.CatBoostClassifier object at 0x000002336A12BB00>,
          fit_params=None, iid='warn', n_iter=10, n_jobs=-1,
          param_distributions={'depth': [6, 8, 10, 12], 'iterations': [1000, 1500, 2000, 2500, 3000, 4000, 5000]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=0)

In [54]:
# report(random_search_2.cv_results_,5)

Model with rank: 1
Mean validation score: 0.765 (std: 0.00300)
Parameters: {'iterations': 3000, 'depth': 12}

Model with rank: 2
Mean validation score: 0.763 (std: 0.00366)
Parameters: {'iterations': 1500, 'depth': 12}

Model with rank: 3
Mean validation score: 0.763 (std: 0.00263)
Parameters: {'iterations': 2500, 'depth': 10}

Model with rank: 4
Mean validation score: 0.761 (std: 0.00324)
Parameters: {'iterations': 1000, 'depth': 12}

Model with rank: 5
Mean validation score: 0.759 (std: 0.00368)
Parameters: {'iterations': 5000, 'depth': 8}



In [77]:
model_best= CatBoostClassifier(class_weights=[1,2],eval_metric='AUC',loss_function="Logloss",random_seed = 23,learning_rate=0.1,iterations=3000,depth=12,metric_period = 100,od_wait=100,od_type='Iter')

In [78]:
model_best.fit(X_train, Y_train,
             eval_set=(X_test,Y_test),
             use_best_model=True,
             verbose=True)

0:	test: 0.6689566	best: 0.6689566 (0)	total: 428ms	remaining: 21m 23s
100:	test: 0.7579262	best: 0.7579262 (100)	total: 39.8s	remaining: 19m 2s
200:	test: 0.7742850	best: 0.7742850 (200)	total: 1m 26s	remaining: 19m 58s
300:	test: 0.7836815	best: 0.7838207 (299)	total: 2m 11s	remaining: 19m 36s
400:	test: 0.7885537	best: 0.7886352 (393)	total: 2m 59s	remaining: 19m 20s
500:	test: 0.7915821	best: 0.7915821 (500)	total: 3m 48s	remaining: 18m 58s
600:	test: 0.7927450	best: 0.7927810 (599)	total: 4m 34s	remaining: 18m 16s
700:	test: 0.7936678	best: 0.7936774 (699)	total: 5m 21s	remaining: 17m 34s
800:	test: 0.7942245	best: 0.7945184 (758)	total: 6m 7s	remaining: 16m 50s
900:	test: 0.7952889	best: 0.7953246 (899)	total: 6m 52s	remaining: 16m 2s
1000:	test: 0.7954453	best: 0.7955401 (965)	total: 7m 40s	remaining: 15m 18s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.7955400742
bestIteration = 965

Shrink model to first 966 iterations.


In [79]:
p2=model_best.predict_proba(X_test)[:,1]

In [80]:
roc_auc_score(Y_test,p2)

0.7955400742217439

In [81]:
sample_submission["is_pass"]=model_best.predict_proba(testing)[:,1]

In [82]:
sample_submission.to_csv("D:/Python/ML Starter Program/sample_submission_cat.csv",header =True, index=False)

### Using this model for final submission ###

In [33]:
# from sklearn.svm import SVC

In [34]:
# params_2 = {'C': [0.1, 1, 10, 100, 1000],  
#               'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
#               'kernel': ['rbf']}

In [35]:
# random_search_3= RandomizedSearchCV(estimator=SVC(), param_distributions = params_2, verbose = 3) 

In [120]:
# random_search_3.fit(X_train,Y_train)

In [121]:
# report(random_search_3.cv_results_,5)

In [39]:
# SVC_best=SVC(kernel='rbf',C=10,gamma=0.01)

In [122]:
# SVC_best.fit(X_train,Y_train)

In [44]:
# p3=SVC_best.predict(X_test)

In [46]:
# roc_auc_score(Y_test,p3)

0.5547483217509058

In [52]:
# sample_submission["is_pass"]=SVC_best.predict(testing)

In [53]:
# sample_submission.to_csv("D:/Python/ML Starter Program/sample_submission.csv",header =True, index=False)

In [167]:
param_dist_lgb = {
              "learning_rate":[0.01,0.05,0.1,0.2],
    "bagging_freq":[3,9,15,17,23,31],
    "max_depth": [2,3,4,5,6,8,10,12],
              "boosting_type":['gbdt','rf','dart'],
 "min_data":[300,400,500,550,650],
               "max_bin":[5,10,12,18,20],
              "is_unbalance":[True , False],
    "lambda_l1":[0,10,20,30,40],
    'bagging_fraction':[0.5,0.7,0.8,0.9]
    
              }

In [168]:
from lightgbm import LGBMClassifier

lgbm = LGBMClassifier(objective='Binary',random_state=5)

In [169]:
n_iter=10

random_search_4=RandomizedSearchCV(lgbm,n_jobs=-1,verbose=2,cv=10,n_iter=n_iter,scoring='roc_auc',
                                 param_distributions=param_dist_lgb)

In [170]:
random_search_4.fit(X_train,Y_train)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    9.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   42.3s finished


RandomizedSearchCV(cv=10, error_score='raise-deprecating',
          estimator=LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_leaves=31, objective='Binary',
        random_state=5, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0),
          fit_params=None, iid='warn', n_iter=10, n_jobs=-1,
          param_distributions={'learning_rate': [0.01, 0.05, 0.1, 0.2], 'bagging_freq': [3, 9, 15, 17, 23, 31], 'max_depth': [2, 3, 4, 5, 6, 8, 10, 12], 'boosting_type': ['gbdt', 'rf', 'dart'], 'min_data': [300, 400, 500, 550, 650], 'max_bin': [5, 10, 12, 18, 20], 'is_unbalance': [True, False], 'lambda_l1': [0, 10, 20, 30, 40], 'bagging_fraction': [0.5, 0.7, 0.8, 0.9]},
          pre_dispatch='2*n_jobs', random_state=None, refit=T

In [171]:
report(random_search_4.cv_results_,5)

Model with rank: 1
Mean validation score: 0.728 (std: 0.00693)
Parameters: {'min_data': 400, 'max_depth': 10, 'max_bin': 20, 'learning_rate': 0.2, 'lambda_l1': 0, 'is_unbalance': False, 'boosting_type': 'dart', 'bagging_freq': 3, 'bagging_fraction': 0.5}

Model with rank: 2
Mean validation score: 0.723 (std: 0.00656)
Parameters: {'min_data': 500, 'max_depth': 12, 'max_bin': 20, 'learning_rate': 0.1, 'lambda_l1': 20, 'is_unbalance': True, 'boosting_type': 'gbdt', 'bagging_freq': 23, 'bagging_fraction': 0.7}

Model with rank: 3
Mean validation score: 0.722 (std: 0.00646)
Parameters: {'min_data': 300, 'max_depth': 12, 'max_bin': 10, 'learning_rate': 0.2, 'lambda_l1': 10, 'is_unbalance': False, 'boosting_type': 'dart', 'bagging_freq': 15, 'bagging_fraction': 0.7}

Model with rank: 4
Mean validation score: 0.719 (std: 0.00627)
Parameters: {'min_data': 300, 'max_depth': 12, 'max_bin': 10, 'learning_rate': 0.2, 'lambda_l1': 10, 'is_unbalance': False, 'boosting_type': 'dart', 'bagging_freq': 1

In [172]:
lgbm_best=LGBMClassifier(min_data=400,max_depth=10,max_bin=20,learning_rate=0.2,lambda_l1=0,is_unbalance=False,bossting_type='dart',bagging_freq=3,bagging_fraction=0.5)

In [173]:
lgbm_best.fit(X_train,Y_train)

LGBMClassifier(bagging_fraction=0.5, bagging_freq=3, boosting_type='gbdt',
        bossting_type='dart', class_weight=None, colsample_bytree=1.0,
        importance_type='split', is_unbalance=False, lambda_l1=0,
        learning_rate=0.2, max_bin=20, max_depth=10, min_child_samples=20,
        min_child_weight=0.001, min_data=400, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [174]:
p4=lgbm_best.predict_proba(X_test)[:,1]

In [175]:
roc_auc_score(Y_test,p4)

0.7322671457607248

In [176]:
sample_submission["is_pass"]=lgbm_best.predict_proba(testing)[:,1]

In [177]:
sample_submission.to_csv("D:/Python/ML Starter Program/sample_submission_LGBM.csv",header =True, index=False)

In [103]:
# from sklearn.neural_network import MLPClassifier

In [104]:
# from sklearn.model_selection import KFold

In [105]:
# hidden_layer_sizes= [(20,10),(20,11),(20,12),(20,13),(20,14)]

In [106]:
# auc_list=[]

In [ ]:
# for h in hidden_layer_sizes:
    
#     nn = MLPClassifier(hidden_layer_sizes=h,activation='logistic')
# #     
#     kf = KFold(n_splits=10)
    
#     xval_err = 0
    
#     for train, test in kf.split(X_train):
        
#         nn.fit(X_train.iloc[train], Y_train.iloc[train])
#         p = nn.predict_proba(X_train.iloc[test])[:,1]
#         xval_err += roc_auc_score(Y_train.iloc[test],p)
    
#     auc_10cv = xval_err/10
#     print(h, ":",auc_10cv)

(20, 10) : 0.7197211724870842


In [114]:
# nn_best=MLPClassifier(hidden_layer_sizes=(20,10),activation='logistic')

In [115]:
# nn_best.fit(X_train,Y_train)

MLPClassifier(activation='logistic', alpha=0.0001, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(20, 10), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [116]:
# p5=nn_best.predict_proba(X_test)[:,1]

In [117]:
# roc_auc_score(Y_test,p5)

0.7140586382227518

In [118]:
# sample_submission["is_pass"]=nn_best.predict_proba(testing)[:,1]

In [119]:
# sample_submission.to_csv("D:/Python/ML Starter Program/sample_submission_nn.csv",header =True, index=False)

In [1]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=5, weights="uniform")
col=['trainee_id','gender','education','city_tier', 'trainee_engagement_rating'] 
X_tr=train[col]
X_test=test[col] 
X_tr=imputer.fit_transform(X_tr)
X_test=imputer.transform(X_test) 
train=train.join(pd.DataFrame({'imp':X_tr[:,- 1]}))
test=test.join(pd.DataFrame({'imp':X_test[:,- 1]}))


SyntaxError: invalid syntax (<ipython-input-1-56c6bda5a8ac>, line 1)

In [2]:
from catboost import CatBoostClassifier,Pool
from sklearn.model_selection import train_test_split
train_y=train.is_pass
train=train.drop(["is_pass"],axis=1)
x_train,x_valid,y_train,y_valid=train_test_split(train,train_y,train_size =0.8)
cat_features=[0,1,3,6] 
train_pool=Pool(train, train_y, cat_features=cat_features)
test_pool=Pool(test, cat_features =cat_features) 
weight=[1, 0.43]
# Imbalance##
model = CatBoostClassifier(
        iterations=5000,logging_level="Silent",early_stopping_rounds=500,
    use_best_model=True,custom_loss=["AUC"], class_weights=weight, 
        eval_metric="AUC")

model.fit(  x_train,y_train, 
        cat_features=cat_features,
        eval_set=(x_valid,y_valid),
        plot=True 
        )
from sklearn.metrics import roc_auc_score
roc_auc_score(y_valid, model.predict_proba(x_valid) [:,1]) 
feature_importances=model.get_feature_importance(train_pool) 
feature_names=train.columns
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    print('{}: {}'.format(name, score))

ModuleNotFoundError: No module named 'jupytertheme'